In [15]:
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset
!unzip -o -q microsoft-catsvsdogs-dataset.zip

import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from PIL import Image

def safe_load_img(img_path, target_size=None):
    try:
        img = Image.open(img_path)
        if target_size:
            img = img.resize(target_size)
        return np.array(img)
    except:
        print(f"Cannot identify image file: {img_path}")
        return None

class SafeImageDataGenerator(ImageDataGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _get_batches_of_transformed_samples(self, index_array):
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=self.dtype)
        batch_y = np.zeros(len(index_array), dtype=self.dtype)
        for i, j in enumerate(index_array):
            fname = self.filenames[j]
            img_path = os.path.join(self.directory, fname)
            img = safe_load_img(img_path, self.target_size)
            if img is not None:
                x = img_to_array(img, data_format=self.data_format)
                x = self.image_data_generator.random_transform(x)
                x = self.image_data_generator.standardize(x)
                batch_x[i] = x
                batch_y[i] = self.classes[j]
            else:
                continue
        return batch_x, batch_y

base_dir = 'PetImages'
train_dir = os.path.join(base_dir, 'Train')
validation_dir = os.path.join(base_dir, 'Validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

os.makedirs(os.path.join(train_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Dog'), exist_ok=True)
os.makedirs(os.path.join(validation_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(validation_dir, 'Dog'), exist_ok=True)

def is_image_valid(image_path):
    img = safe_load_img(image_path)
    return img is not None

def split_and_move_images(animal):
    src_dir = os.path.join(base_dir, animal)
    train_dest_dir = os.path.join(train_dir, animal)
    val_dest_dir = os.path.join(validation_dir, animal)

    images = [img for img in os.listdir(src_dir) if is_image_valid(os.path.join(src_dir, img))]
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    for img in train_images:
        shutil.move(os.path.join(src_dir, img), os.path.join(train_dest_dir, img))

    for img in val_images:
        shutil.move(os.path.join(src_dir, img), os.path.join(val_dest_dir, img))

split_and_move_images('Cat')
split_and_move_images('Dog')

BATCH_SIZE = 32
IMG_SIZE = (150, 150)

train_datagen = SafeImageDataGenerator(rescale=1./255,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')

validation_datagen = SafeImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=IMG_SIZE,
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='binary')

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

EPOCHS = 10 #100
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=validation_generator,
                    callbacks=[early_stopping])

loss, accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy: {accuracy:.2f}")

model.save('cat_dog_classifier.h5')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
microsoft-catsvsdogs-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Cannot identify image file: PetImages/Cat/Thumbs.db
Cannot identify image file: PetImages/Cat/666.jpg
Cannot identify image file: PetImages/Dog/Thumbs.db


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Cannot identify image file: PetImages/Dog/11702.jpg
Found 21685 images belonging to 2 classes.
Found 6687 images belonging to 2 classes.
Epoch 1/10
678/678 [==============================] - 194s 283ms/step - loss: 0.6562 - accuracy: 0.5985 - val_loss: 0.6950 - val_accuracy: 0.5565
Epoch 2/10
678/678 [==============================] - 191s 282ms/step - loss: 0.5889 - accuracy: 0.6829 - val_loss: 0.5364 - val_accuracy: 0.7356
Epoch 3/10
678/678 [==============================] - 189s 279ms/step - loss: 0.5468 - accuracy: 0.7195 - val_loss: 0.4972 - val_accuracy: 0.7648
Epoch 4/10
678/678 [==============================] - 189s 278ms/step - loss: 0.5116 - accuracy: 0.7451 - val_loss: 0.4705 - val_accuracy: 0.7790
Epoch 5/10
678/678 [==============================] - 189s 278ms/step - loss: 0.4908 - accuracy: 0.7618 - val_loss: 0.4183 - val_accuracy: 0.8108
Epoch 6/10
678/678 [==============================] - 188s 278ms/step - loss: 0.4736 - accuracy: 0.7732 - val_loss: 0.4103 - val_accu

In [12]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_image(image_path, model):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = img_array / 255.
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    return prediction

model = load_model('cat_dog_classifier.h5')

image_path = "/content/puma.jpeg"
prediction = predict_image(image_path, model)

if prediction[0] < 0.5:
    print("Predicción: Gato")
else:
    print("Predicción: Perro")


1/1 [==============================] - 0s 77ms/step
Predicción: Gato


In [14]:
from sklearn.metrics import confusion_matrix

# Obtener las etiquetas verdaderas y las predicciones en el conjunto de validación
y_true = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred_binary = np.round(y_pred).flatten()

# Calcular la matriz de confusión
cm = confusion_matrix(y_true, y_pred_binary)

# Imprimir la matriz de confusión
print(cm)




209/209 [==============================] - 20s 98ms/step
[[2737 1450]
 [1615  885]]
La clase minoritaria es Perro


In [ ]:

(unique_labels, counts) = np.unique(y_true, return_counts=True)

# Encontrar la clase minoritaria
minority_class_index = np.argmin(counts)
minority_class = unique_labels[minority_class_index]

if minority_class == 0:
    print("La clase minoritaria es Gato")
else:
    print("La clase minoritaria es Perro")